In [1]:
#Importing Libraries !!
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split  
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn import metrics
from sklearn import tree
from sklearn.tree import export_graphviz

import matplotlib.pyplot as plt  
%matplotlib inline
import seaborn as sn

import scipy.stats.stats as stats


from sklearn.model_selection import GridSearchCV  


# import scikitplot as skplt
import pickle
from sklearn.externals import joblib

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTEENN

from imblearn import under_sampling, over_sampling, combine

Using TensorFlow backend.


In [2]:
df1=pd.read_csv("state_wise_daily.csv",parse_dates=[0])

In [3]:
df1 = df1.melt(id_vars=['Date','Status'])

In [4]:
df1 = df1.rename(columns = {"variable" : "State", "value" : "Cases"})

In [5]:
df2 = df1.sort_values(by=['Date', 'Status'])

In [6]:
df3=df2[df2["Status"]=="Confirmed"]
df3 = df3[df3["State"] != "TT"]

In [7]:
df3 = df3.rename(columns = {"Cases" : "n_Confirmed"})
df3 = df3[["Date","State","n_Confirmed"]]

In [144]:
df3.to_csv("cases.csv")

In [8]:
df4 = df2[df2["Status"]=="Deceased"]
df4 = df4[df4["State"] != "TT"]

df4 = df4.rename(columns = {"Cases" : "n_Deceased"})
df4 = df4[["Date","State","n_Deceased"]]

In [14]:
df5 = df2[df2["Status"]=="Recovered"]
df5 = df5[df5["State"] != "TT"]

df5 = df5.rename(columns = {"Cases" : "n_Recovered"})
df5 = df5[["Date","State","n_Recovered"]]

In [15]:
df6 = pd.merge(df3, df4, how = "left", left_on=["Date","State"], right_on = ["Date","State"])

In [16]:
df7 = pd.merge(df6, df5, how = "left", left_on=["Date","State"], right_on = ["Date","State"])

In [18]:
df8 = df7[["n_Confirmed", "n_Deceased", "n_Recovered"]]

In [19]:
df8 = df8.cumsum()

In [20]:
df8 = df8.rename(columns = {"n_Confirmed" : "Cum_Confirmed", "n_Deceased" : "Cum_Deceased", "n_Recovered" : "Cum_Recovered"} )

In [21]:
df8 = pd.concat([df7, df8], axis=1)

In [22]:
df8["n_Active"] = df8.Cum_Confirmed - df8.Cum_Deceased -df8.Cum_Recovered

In [25]:
df22=df8
df23 =df8
df23["perc_Active"] = df22.n_Active/df22.Cum_Confirmed
df23["perc_Deceased"] = df22.Cum_Deceased/df22.Cum_Confirmed
df23["perc_Recovered"] = df22.Cum_Recovered/df22.Cum_Confirmed

In [26]:
df24=df8
df24["State_Cum_Confirmed"] = df23.groupby("State")["n_Confirmed"].cumsum()
df24["State_Cum_Deceased"] = df23.groupby("State")["n_Deceased"].cumsum()
df24["State_Cum_Recovered"] = df23.groupby("State")["n_Recovered"].cumsum()
df24["State_n_Active"] = df24.State_Cum_Confirmed - df24.State_Cum_Deceased - df24.State_Cum_Recovered
df24["State_perc_Active"] = df22.State_n_Active/df22.State_Cum_Confirmed
df24["State_perc_Deceased"] = df22.State_Cum_Deceased/df22.State_Cum_Confirmed
df24["State_perc_Recovered"] = df22.State_Cum_Recovered/df22.State_Cum_Confirmed


In [27]:
district=pd.read_csv("districts.csv",parse_dates=[0])
tier_data = pd.read_csv("Tier_Data.csv")

In [28]:
district_data = pd.merge(district, tier_data, how = "left", left_on=["State", "District"], right_on = ["State", "District"])

In [29]:
district_data_summ1 = pd.pivot_table(district_data, values = ["Confirmed","Deceased","Recovered"], index = ["Date", "State_Code"], columns = ["Tier"], aggfunc = np.sum)

In [30]:
district_data_summ1.columns =[s1 + str(s2) for (s1,s2) in district_data_summ1.columns.tolist()]

In [31]:
district_data_summ1.reset_index(inplace=True)

In [32]:
df25 = pd.merge(df24, district_data_summ1, how = "left", left_on=["Date", "State"], right_on = ["Date", "State_Code"])

In [33]:
df26 = df25.drop('State_Code', axis=1)

In [34]:
df27 = df26.replace(r'^\s*$', np.nan, regex=True)
df27.update(df27[["n_Confirmed","n_Deceased","n_Recovered","Cum_Confirmed","Cum_Deceased","Cum_Recovered","n_Active","perc_Active","perc_Deceased","perc_Recovered","State_Cum_Confirmed","State_Cum_Deceased","State_Cum_Recovered","State_n_Active","State_perc_Active","State_perc_Deceased","State_perc_Recovered","Confirmed Tier-1 ","Confirmed Tier-2 ","Confirmed Tier-3 ","Deceased Tier-1 ","Deceased Tier-2 ","Deceased Tier-3 ","Recovered Tier-1 ","Recovered Tier-2 ","Recovered Tier-3 "]].fillna(0))

In [35]:
df28 = df27.rename(columns = {"Confirmed Tier-1 " : "State_Cum_Confirmed_Tier1","Confirmed Tier-2 ": "State_Cum_Confirmed_Tier2","Confirmed Tier-3 ": "State_Cum_Confirmed_Tier3"
                             , "Deceased Tier-1 " : "State_Cum_Deceased_Tier1","Deceased Tier-2 ":"State_Cum_Deceased_Tier2","Deceased Tier-3 ":"State_Cum_Deceased_Tier2"
                             , "Recovered Tier-1 ": "State_Cum_Recovered_Tier1","Recovered Tier-2 ":"State_Cum_Recovered_Tier2","Recovered Tier-3 ":"State_Cum_Recovered_Tier3"}, inplace=True)

In [36]:
# df27.to_csv("df27.csv")

In [37]:
df27=df27.sort_values(by=['State','Date'], ascending=True)

In [38]:
df27.reset_index(inplace=True,drop=True)

In [39]:
df27["n_Confirmed_d_1"]=df27.groupby(['State'])['n_Confirmed'].shift(1)
df27["n_Confirmed_d_2"]=df27.groupby(['State'])['n_Confirmed'].shift(2)
df27["n_Confirmed_d_3"]=df27.groupby(['State'])['n_Confirmed'].shift(3)
df27["n_Confirmed_d_4"]=df27.groupby(['State'])['n_Confirmed'].shift(4)
df27["n_Confirmed_d_5"]=df27.groupby(['State'])['n_Confirmed'].shift(5)
df27["n_Confirmed_d_6"]=df27.groupby(['State'])['n_Confirmed'].shift(6)
df27["n_Confirmed_d_7"]=df27.groupby(['State'])['n_Confirmed'].shift(7)
df27["n_Confirmed_d_8"]=df27.groupby(['State'])['n_Confirmed'].shift(8)
df27["n_Confirmed_d_9"]=df27.groupby(['State'])['n_Confirmed'].shift(9)
df27["n_Confirmed_d_10"]=df27.groupby(['State'])['n_Confirmed'].shift(10)
df27["n_Confirmed_d_11"]=df27.groupby(['State'])['n_Confirmed'].shift(11)
df27["n_Confirmed_d_12"]=df27.groupby(['State'])['n_Confirmed'].shift(12)
df27["n_Confirmed_d_13"]=df27.groupby(['State'])['n_Confirmed'].shift(13)
df27["n_Confirmed_d_14"]=df27.groupby(['State'])['n_Confirmed'].shift(14)
df27["n_Confirmed_d_15"]=df27.groupby(['State'])['n_Confirmed'].shift(15)
df27["n_Confirmed_d_16"]=df27.groupby(['State'])['n_Confirmed'].shift(16)
df27["n_Confirmed_d_17"]=df27.groupby(['State'])['n_Confirmed'].shift(17)
df27["n_Confirmed_d_18"]=df27.groupby(['State'])['n_Confirmed'].shift(18)
df27["n_Confirmed_d_19"]=df27.groupby(['State'])['n_Confirmed'].shift(19)
df27["n_Confirmed_d_20"]=df27.groupby(['State'])['n_Confirmed'].shift(20)
df27["n_Confirmed_d_21"]=df27.groupby(['State'])['n_Confirmed'].shift(21)
df27["n_Confirmed_d_22"]=df27.groupby(['State'])['n_Confirmed'].shift(22)
df27["n_Confirmed_d_23"]=df27.groupby(['State'])['n_Confirmed'].shift(23)
df27["n_Confirmed_d_24"]=df27.groupby(['State'])['n_Confirmed'].shift(24)
df27["n_Confirmed_d_25"]=df27.groupby(['State'])['n_Confirmed'].shift(25)
df27["n_Confirmed_d_26"]=df27.groupby(['State'])['n_Confirmed'].shift(26)
df27["n_Confirmed_d_27"]=df27.groupby(['State'])['n_Confirmed'].shift(27)
df27["n_Confirmed_d_28"]=df27.groupby(['State'])['n_Confirmed'].shift(28)
df27["n_Confirmed_d_29"]=df27.groupby(['State'])['n_Confirmed'].shift(29)
df27["n_Confirmed_d_30"]=df27.groupby(['State'])['n_Confirmed'].shift(30)

df27["n_Confirmed_d+1"]=df27.groupby(['State'])['n_Confirmed'].shift(-1)
df27["n_Confirmed_d+2"]=df27.groupby(['State'])['n_Confirmed'].shift(-2)
df27["n_Confirmed_d+3"]=df27.groupby(['State'])['n_Confirmed'].shift(-3)
df27["n_Confirmed_d+4"]=df27.groupby(['State'])['n_Confirmed'].shift(-4)
df27["n_Confirmed_d+5"]=df27.groupby(['State'])['n_Confirmed'].shift(-5)
df27["n_Confirmed_d+6"]=df27.groupby(['State'])['n_Confirmed'].shift(-6)
df27["n_Confirmed_d+7"]=df27.groupby(['State'])['n_Confirmed'].shift(-7)
df27["n_Confirmed_d+8"]=df27.groupby(['State'])['n_Confirmed'].shift(-8)
df27["n_Confirmed_d+9"]=df27.groupby(['State'])['n_Confirmed'].shift(-9)
df27["n_Confirmed_d+10"]=df27.groupby(['State'])['n_Confirmed'].shift(-10)
df27["n_Confirmed_d+11"]=df27.groupby(['State'])['n_Confirmed'].shift(-11)
df27["n_Confirmed_d+12"]=df27.groupby(['State'])['n_Confirmed'].shift(-12)
df27["n_Confirmed_d+13"]=df27.groupby(['State'])['n_Confirmed'].shift(-13)
df27["n_Confirmed_d+14"]=df27.groupby(['State'])['n_Confirmed'].shift(-14)
df27["n_Confirmed_d+15"]=df27.groupby(['State'])['n_Confirmed'].shift(-15)
df27["n_Confirmed_d+16"]=df27.groupby(['State'])['n_Confirmed'].shift(-16)
df27["n_Confirmed_d+17"]=df27.groupby(['State'])['n_Confirmed'].shift(-17)
df27["n_Confirmed_d+18"]=df27.groupby(['State'])['n_Confirmed'].shift(-18)
df27["n_Confirmed_d+19"]=df27.groupby(['State'])['n_Confirmed'].shift(-19)
df27["n_Confirmed_d+20"]=df27.groupby(['State'])['n_Confirmed'].shift(-20)
df27["n_Confirmed_d+21"]=df27.groupby(['State'])['n_Confirmed'].shift(-21)





In [40]:
variables = ['n_Deceased', 'n_Recovered',
       'Cum_Confirmed', 'Cum_Deceased', 'Cum_Recovered', 'n_Active',
       'perc_Active', 'perc_Deceased', 'perc_Recovered', 'State_Cum_Confirmed',
       'State_Cum_Deceased', 'State_Cum_Recovered', 'State_n_Active',
       'State_perc_Active', 'State_perc_Deceased', 'State_perc_Recovered',
       'State_Cum_Confirmed_Tier1', 'State_Cum_Confirmed_Tier2',
       'State_Cum_Confirmed_Tier3', 'State_Cum_Deceased_Tier1',
       'State_Cum_Deceased_Tier2', 'State_Cum_Deceased_Tier2',
       'State_Cum_Recovered_Tier1', 'State_Cum_Recovered_Tier2',
       'State_Cum_Recovered_Tier3']

In [ ]:
for i in variables:
    for y in range(1,31):
        var=str(i)+"_"+str(y)
        df27[var]=df27.groupby(['State'])[i].shift(y)

In [42]:
df27.shape

(5244, 679)

In [43]:
df27=df27.drop(variables, axis=1)

In [44]:
df27=df27.drop("n_Confirmed",axis=1)

In [118]:
df27.shape

(5244, 653)

In [119]:
# pd.DataFrame(df27.columns).to_csv("columns.csv")

In [120]:
# df27.to_csv("train_data.csv")

In [121]:
train_data=df27[df27["Date"]>="2020-05-01"]
train_data=train_data[train_data["Date"]<"2020-07-08"]


In [122]:
Ys=['n_Confirmed_d+1',
'n_Confirmed_d+2',
'n_Confirmed_d+3',
'n_Confirmed_d+4',
'n_Confirmed_d+5',
'n_Confirmed_d+6',
'n_Confirmed_d+7',
'n_Confirmed_d+8',
'n_Confirmed_d+9',
'n_Confirmed_d+10',
'n_Confirmed_d+11',
'n_Confirmed_d+12',
'n_Confirmed_d+13',
'n_Confirmed_d+14',
'n_Confirmed_d+15',
'n_Confirmed_d+16',
'n_Confirmed_d+17',
'n_Confirmed_d+18',
'n_Confirmed_d+19',
'n_Confirmed_d+20',
'n_Confirmed_d+21'
]

In [123]:
Y=train_data[Ys]
X=train_data.drop(Ys, axis=1)
X=X.drop(["Date","State"], axis=1)


### XGB

In [124]:
from sklearn.multioutput import MultiOutputRegressor

In [125]:
from xgboost import XGBRegressor

In [126]:
wrapper = MultiOutputRegressor(XGBRegressor(n_estimators=200,max_depth=2))

In [127]:
wrapper.fit(X, Y)

[21:38:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:39:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:39:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:39:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:39:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:40:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:40:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:40:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=2, min_child_weight=1,
                                            missing=None, n_estimators=200,
                                            n_jobs=1, nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbos

## Predictions

In [128]:
predict_data=df27[df27["Date"]=="2020-07-29"]
predict_X=predict_data.drop(Ys,axis=1)
predict_X=predict_X.drop(["Date","State"], axis=1)

In [129]:
predict_Y=wrapper.predict(predict_X)

In [130]:
predict_dates=pd.date_range("07-30-2020", "08-19-2020")

In [131]:
Predictions=pd.DataFrame(data=predict_Y, index=predict_data["State"],columns=predict_dates)

In [132]:
Predictions

,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,...,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19
State,,,,,,,,,,,,,,,,,,,,,
AN,20.435537,-17.619904,-137.666275,87.099541,-58.648155,53.427227,79.284676,106.815849,171.255844,58.624268,...,59.382790,27.545399,30.047941,51.409027,43.379269,50.691246,46.127518,-551.648376,47.867001,82.795937
AP,3425.982910,3107.333984,2598.490723,3010.917480,3323.386475,3304.681396,2873.196289,2939.127197,3445.666260,3811.210693,...,4850.651367,5732.716797,5593.375977,6098.499512,7466.567871,8149.505859,8894.184570,7614.767578,7808.497070,7556.787598
AR,59.524708,26.656143,-99.680458,100.438866,-17.488930,88.166130,128.311310,129.152252,210.926468,94.494209,...,92.370804,101.859604,87.233391,117.567741,95.913071,125.190285,97.167725,-494.478607,94.649261,140.539474
AS,1593.375854,1761.163330,1434.629517,1684.186401,1787.781494,1921.081909,1942.170898,2113.657227,2583.449951,2115.939697,...,2027.573975,2332.839844,1938.259399,2850.081543,3384.978271,3552.424316,3531.181885,3649.545166,4960.138184,5571.786621
BR,2054.245117,1961.051880,1881.716797,2061.740479,2334.236816,2239.480225,2517.212402,2625.119385,2777.476807,2735.180176,...,3012.928711,3761.391602,3563.159912,4157.653809,5470.176270,5950.958984,6258.013184,5798.299316,6114.643066,5695.247070
CH,32.377346,-16.430252,-126.074097,110.993874,-43.891029,62.851742,111.775360,137.070465,208.271042,76.207848,...,87.678520,79.986839,74.277039,88.805115,95.665733,100.601089,98.781021,-502.028290,124.514214,128.564148
CT,362.207947,433.046936,134.418930,451.103271,378.443695,426.479401,434.656250,573.380188,808.806213,584.911743,...,602.246826,617.744629,632.524414,786.852356,769.475464,812.707642,768.504761,270.515991,946.786438,1584.270874
DD,0.248157,-34.383480,-159.356140,57.820122,-77.962204,30.737164,60.323593,72.629593,153.680847,39.086983,...,19.814322,7.471912,6.127644,29.313725,16.798304,35.438347,21.162170,-623.840393,16.464619,51.593655
DL,1657.148438,1650.118042,1707.589966,1718.274902,1529.434692,1612.839722,1887.456299,2602.213135,2493.184570,2908.288818,...,2356.482178,3119.386475,2076.298340,2745.574463,2074.653564,2672.936768,2237.906982,1864.634888,1822.891479,2062.101807


In [133]:
Predictions1 = Predictions.reset_index()

In [134]:
Predictions1=Predictions1.melt(id_vars="State")

In [135]:
Predictions1.to_csv("Multioutput_xgb_fut.csv")

### Multiple XGB in Loop

In [136]:
model=XGBRegressor(n_estimators=200,max_depth=2)

In [137]:
p1=df27.shape

In [138]:
p2=df27.iloc[p1[0]-1]["Date"]

In [139]:

train_data=df27[df27["Date"]>="2020-05-01"]

cols=["Date","State","predict_case"]

Final_Predict=pd.DataFrame(columns=cols)

Final_Predict_hist=Final_Predict

j=-1
for i in Ys:
    p3=p2+pd.DateOffset(days=j)
    train1=train_data[train_data["Date"]<=p3]
    Y=train1[i]
    X=train1.drop(Ys, axis=1)
    X=X.drop(["Date","State"], axis=1)
    model.fit(X,Y)
    
    
    predict_data=df27[df27["Date"]==p2]
    predict_X=predict_data.drop(Ys,axis=1)
    predict_X=predict_X.drop(["Date","State"], axis=1)
    predict_Y=model.predict(predict_X)
    Predictions=pd.DataFrame(data=predict_Y,columns=["predict_case"])
    predict_data.reset_index(inplace=True, drop=True)
    Predictions["State"]=predict_data["State"]
    Predictions["Date"]=predict_data.loc[:,"Date"]+pd.DateOffset(days=-j)
    Predictions=Predictions[cols]
        
    Final_Predict=Final_Predict.append(Predictions)
    
    predict_data1=df27[df27["Date"]=="07-20-2020"]
    predict_X1=predict_data1.drop(Ys,axis=1)
    predict_X1=predict_X1.drop(["Date","State"], axis=1)
    predict_Y1=model.predict(predict_X1)
    Predictions1=pd.DataFrame(data=predict_Y1,columns=["predict_case"])
    predict_data1.reset_index(inplace=True, drop=True)
    Predictions1["State"]=predict_data1["State"]
    Predictions1["Date"]=predict_data1.loc[:,"Date"]+pd.DateOffset(days=-j)
    Predictions1=Predictions1[cols]
        
    Final_Predict_hist=Final_Predict_hist.append(Predictions1)  
    
    j=j-1
    

[21:44:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:44:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:44:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:45:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:46:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [140]:
Final_Predict

,Date,State,predict_case
0,2020-07-30,AN,6.024548
1,2020-07-30,AP,8523.401367
2,2020-07-30,AR,46.957001
3,2020-07-30,AS,1327.623169
4,2020-07-30,BR,2467.486816
...,...,...,...
33,2020-08-19,TR,751.749451
34,2020-08-19,UN,60.908642
35,2020-08-19,UP,6576.984863
36,2020-08-19,UT,1609.473511


In [141]:
Final_Predict_hist

,Date,State,predict_case
0,2020-07-21,AN,13.783671
1,2020-07-21,AP,5513.478516
2,2020-07-21,AR,49.696289
3,2020-07-21,AS,1314.137573
4,2020-07-21,BR,1528.200439
...,...,...,...
33,2020-08-10,TR,397.081055
34,2020-08-10,UN,35.663593
35,2020-08-10,UP,4911.714355
36,2020-08-10,UT,542.891357


In [142]:
Final_Predict.to_csv("multi_xgb_fut.csv")
Final_Predict_hist.to_csv("multi_xgb_hist.csv")